Осоргин Иван БКЛ211

In [ ]:
!pip3 install matplotlib transformers ipywidgets pandas
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [ ]:
import csv

filename = 'CommitmentBank-items.csv'
with open(filename, 'r') as f:
    colnames = next(csv.reader(f))

def rows_gen():
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        _ = next(reader)
        for row in reader:
            yield {k: v for k, v in zip(colnames, row)}

## Задача 1

In [ ]:
from collections import defaultdict
from typing import Tuple, Dict
predicates = {}
categories = ['modal', 'conditional', 'negation', 'question']
# counting occurances
for r in rows_gen():
    if not r['Verb'] in predicates:
        predicates[r['Verb']] = defaultdict(list)
    for category in categories:
        if category in r['Embedding']:
            predicates[r['Verb']][category].append(float(r['Mean']))
# counting average for each
for p in predicates.keys():
    for c in categories:
        l = len(predicates[p][c])
        if l == 0: 
            del predicates[p][c]
            continue
        predicates[p][c] = (sum(predicates[p][c]) / l, l)
# restructuring data to list of tuples
predicates = [(k, dict(v)) for k, v in predicates.items()]
# filtering out predicates that have less than `treshold` occurences
def enough(predicate: Tuple[str, Dict[str, Tuple[float, int]]]) -> bool:
    treshold = 2
    for _, it in predicate[1].items():
        if it[1] < treshold:
            return False
    return True
print(f"Size before: {len(predicates)};", end=' ')
predicates = [p for p in predicates if enough(p)]
print(f"after: {len(predicates)}")
# sorting by mean modality
predicates.sort(key=lambda x: sum([i[0] for i in x[1].values()]) / len(x[1]))
predicates

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(10,6))
plt.xticks(ticks=[i for i in range(len(predicates))],
           labels=[p[0] for p in predicates],
           rotation=50)
plt.ylim(-3.3, 3.3)

colors = ['r', 'g', 'b', 'black']
markers = ['$M$', '$C$', '$N$', '$Q$']
for cat, col, mar in zip(categories, colors, markers):
    y_data = [it[cat][0] if cat in it else 0 for _, it in predicates]
    plt.plot(y_data, color=col, marker=mar, linewidth=1, markersize=9)

plt.show()

## Задача 2

| Источник | Предикат | Тип | Контекст | Цель | Рейтинг |
|---|---|---|---|---|---|
| НКРЯ \| [Наши дети: Подростки (2004)] | решить | ничего | Но дело в том, что если в книге будет рассказано в таком стиле, вроде сказочных персонажей, объясняющих как и что надо делать, думаю меня ребенок просто засмеет.  Сейчас другие приоритеты. | Он решит, что я свихнулась. | -2 |
| НКРЯ \| [Запись LiveJournal (2004)] | сказать | ничего | Еще сегодня один знакомый из Туниса навел меня на интересную мысль по поводу манер. | Он в разговоре сказал, что всегда открывает девушке дверь машины и помогает сесть, но при этом в душе он ждет, что пока он обходит машину, она изнутри тоже откроет ему дверь, по крайней мере приоткроет. | 2 |
| НКРЯ \| [Андрей Митьков. Мороз по коже. Этап Кубка мира по лыжам в Токсове провели в экстремальных погодных условиях // «Известия», 2003.01.08] | знать | отрицание | Однако на старт в Токсове вышли только три суперзвезды из шести заявленных. | Организаторы не знали, что Бельмондо торжественно проводили из спорта еще весной 2002 года. | 3

## Задача 3

In [ ]:
# Preparing NLI dataset
with open('dataset.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(('sentence', 'label'))
    for r in rows_gen():
        mean = float(r['Mean'])
        if mean <= -1: label = 'contradiction'
        elif mean >= 1: label = 'entailment'
        else: label = 'neutral'
        writer.writerow((
            f"{r['Context']} {r['Target']}",
            label
        ))

In [ ]:
from transformers import BertTokenizer, BertModel
from torch import nn, argmax, no_grad
import pandas as pd
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset, random_split
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
num_labels = 3
classifier = nn.Linear(768, num_labels)
model = nn.Sequential(model, classifier)
criterion = nn.CrossEntropyLoss()  
optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
class CsvDataSet(Dataset):
    def __init__(self, csv_file, transform=None):
        self.df = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        return self.df.sentence[index], self.df.label[index]
data_set = CsvDataSet('dataset.csv')
train_size = int(0.8 * len(data_set))
test_size = len(data_set) - train_size
train_set, test_set = random_split(data_set, [train_size, test_size])
print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(test_size))

In [ ]:
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
test_loader = DataLoader(test_set, batch_size=16)

In [ ]:
def train(model, optimizer, train_loader, criterion):
    model.train()
    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()
        sentences, labels = batch
        encoded_input = tokenizer(sentences, return_tensors='pt')
        outputs = model(encoded_input)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Training loss: {total_loss/len(train_loader)}')

def evaluate(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    total_acc = 0

    with no_grad():
        for batch in test_loader:
            sentences, labels = batch
            outputs = model(sentences)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            predictions = argmax(outputs, dim=1)
            total_acc += (predictions == labels).sum().item()

    print(f'Test loss: {total_loss/len(test_loader)} Test acc: {total_acc/len(test_set)*100}%')

In [ ]:
encoded_input = tokenizer("Hello I'm a [MASK] model.", return_tensors='pt')
outputs = model(**encoded_input)
outputs

In [ ]:
for epoch in range(3):
    train(model, optimizer, train_loader, criterion)
    evaluate(model, test_loader, criterion)